In [1]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

from ema_workbench import perform_experiments
from ema_workbench import Samplers
from ema_workbench.analysis import prim
from ema_workbench import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Create an instance of the model which is usable with the EMA workbench
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [3]:
zero_policy = {"DaysToThreat": 0}
zero_policy.update({f"DikeIncrease {n}": 0 for n in planning_steps})
zero_policy.update({f"RfR {n}": 0 for n in planning_steps})

pol0 = {}

for key in dike_model.levers:
    s1, s2 = key.name.split("_")
    pol0.update({key.name: zero_policy[s2]})

policy0 = Policy("Policy 0", **pol0)

In [4]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=2000,policies=policy0,uncertainty_sampling=Samplers.LHS)

experiments, outcomes = results

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
100%|██████████████████████████████████████| 2000/2000 [03:26<00:00,  9.71it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [5]:
df_outcomes = pd.DataFrame(outcomes)
df_outcomes["Expected Number of Deaths"] = df_outcomes["A.1_Expected Number of Deaths"] + df_outcomes["A.2_Expected Number of Deaths"] + df_outcomes["A.3_Expected Number of Deaths"] + df_outcomes["A.4_Expected Number of Deaths"] + df_outcomes["A.5_Expected Number of Deaths"]
df_outcomes.to_csv(r'C:\Users\simon\PycharmProjects\EPA141A---MD\final assignment\Results\Outcomes_PRIM.csv')
df_experiments = pd.DataFrame(experiments)
df_experiments.to_csv(r'C:\Users\simon\PycharmProjects\EPA141A---MD\final assignment\Results\Experiments_PRIM.csv')

In [6]:
x = experiments.iloc[:, :19]
y = ((df_outcomes["A.5_Expected Number of Deaths"] >= list(np.sort(df_outcomes["A.5_Expected Number of Deaths"]))[int(0.80* len(df_outcomes["A.5_Expected Number of Deaths"]))]) & (df_outcomes["Expected Number of Deaths"] >= list(np.sort(df_outcomes["Expected Number of Deaths"]))[int(0.80* len(df_outcomes["Expected Number of Deaths"]))]))

In [7]:
prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha=0.1)
box1 = prim_alg.find_box()
box1.show_tradeoff(annotated=True)
plt.show()

[MainProcess/INFO] 2000 points remaining, containing 34 cases of interest


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
box1.select(0)
box1.show_pairs_scatter()
fig = plt.gcf()
fig.set_size_inches(12,12)
plt.show()